In [23]:
import torch
import torch.nn.functional as F  
from torch import optim 
from torch import nn
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import torch.nn.init as init
import random
import time
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from nn_implementation import *

## Function for Grid Search

In [24]:
def grid_search_cv(hidden_layer_sizes_list, activation_functions, learning_rates, batch_sizes, num_epochs_list, train_loader, NumbOfClasses, k_folds=5, use_scaling=True):
    best_f1 = 0.0
    best_combination = None
    results = []

    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    if use_scaling:
        scaler = StandardScaler()
        scaler.fit(train_loader.dataset.tensors[0].numpy())

    for hidden_layer_sizes in hidden_layer_sizes_list:
        for activation_function in activation_functions:
            for learning_rate in learning_rates:
                for batch_size in batch_sizes:
                    for num_epochs in num_epochs_list:

                        fold_accuracies = []
                        fold_f1s = []
                        fold_training_times = []

                        for train_index, test_index in kf.split(train_loader.dataset):
                            X_train, X_test = train_loader.dataset.tensors[0][train_index], train_loader.dataset.tensors[0][test_index]
                            y_train, y_test = train_loader.dataset.tensors[1][train_index], train_loader.dataset.tensors[1][test_index]

                            model = NN(input_size=train_loader.dataset.tensors[0].shape[1],
                                       num_classes=NumbOfClasses,
                                       hidden_layer_sizes=hidden_layer_sizes,
                                       activation_function=activation_function)
                            criterion = nn.CrossEntropyLoss()
                            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

                        
                            if use_scaling:
                                X_train_scaled = torch.tensor(scaler.transform(X_train.numpy()))
                                X_test_scaled = torch.tensor(scaler.transform(X_test.numpy()))
                            else:
                                X_train_scaled, X_test_scaled = X_train, X_test

                            train_start_time = time.time()
                            train_model(model, DataLoader(TensorDataset(X_train_scaled, y_train), batch_size=batch_size, shuffle=True), optimizer, criterion, num_epochs)
                            train_end_time = time.time()
                            fold_training_times.append(train_end_time - train_start_time)

                            accuracy_test, f1_test = check_accuracy(DataLoader(TensorDataset(X_test_scaled, y_test), batch_size=batch_size, shuffle=False), model)
                            fold_accuracies.append(accuracy_test.item())
                            fold_f1s.append(f1_test)

                        avg_accuracy = np.mean(fold_accuracies)
                        avg_f1 = np.mean(fold_f1s)
                        avg_training_time = np.mean(fold_training_times)

                        result = {
                            'Hidden Layer Sizes': hidden_layer_sizes,
                            'Activation Function': activation_function.__name__,
                            'Learning Rate': learning_rate,
                            'Batch Size': batch_size,
                            'Number of Epochs': num_epochs,
                            'Average Accuracy': avg_accuracy,
                            'Average F1': avg_f1,
                            'Average Training Time': avg_training_time
                        }

                        results.append(result)

                        if avg_f1 > best_f1:
                            best_f1 = avg_f1
                            best_combination = result

    results_df = pd.DataFrame(results)
    return results_df, best_f1, best_combination

## Testing the model on the wine quality dataset

#### Loading the dataset

In [25]:
wine_quality = pd.read_csv('./preprocessed-datasets/wine_quality_prepro.csv', index_col=0)
wine_quality.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,class,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


#### Splitting the dataset into training and testing sets, converting to PyTorch tensors and creating PyTorch DataLoaders

In [26]:
train_X, train_Y, test_X, test_Y = train_test_split(wine_quality, "class", return_torch=True)

dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=False)

dataset = TensorDataset(test_X, test_Y)
test_loader = DataLoader(dataset, batch_size=32, shuffle=False)


#### Creating the model, training and testing

In [27]:
input_size = train_X.shape[1] # number of features in wine quality dataset
NumbOfClasses = 10 # 10 classes in wine quality dataset
learning_rate = 0.01
batch_size = 32
num_epochs = 10
hidden_layer_sizes = [25,30]
activation_function = F.tanh

model = NN(input_size=train_X.shape[1], num_classes=NumbOfClasses, hidden_layer_sizes=hidden_layer_sizes, activation_function=activation_function)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_model(model, train_loader, optimizer, criterion, num_epochs)
accuracy, f1 = check_accuracy(train_loader, model)
print(f"Training set accuracy: {accuracy}, F1 Score: {f1}")
accuracy, f1 = check_accuracy(test_loader, model)
print(f"Test set accuracy: {accuracy}, F1 Score: {f1}")

Epoch 1/25, Average Loss: 1.333803896523692
Epoch 2/25, Average Loss: 1.2906058975523966
Epoch 3/25, Average Loss: 1.2820148658167365
Epoch 4/25, Average Loss: 1.2806553131232232
Epoch 5/25, Average Loss: 1.2760694824113437
Epoch 6/25, Average Loss: 1.2697672090647412
Epoch 7/25, Average Loss: 1.2612591437035543
Epoch 8/25, Average Loss: 1.2699610020485392
Epoch 9/25, Average Loss: 1.2569536347330714
Epoch 10/25, Average Loss: 1.257002458981941
Epoch 11/25, Average Loss: 1.2446364515398178
Epoch 12/25, Average Loss: 1.2357006083968227
Epoch 13/25, Average Loss: 1.234742837815197
Epoch 14/25, Average Loss: 1.2384484194539076
Epoch 15/25, Average Loss: 1.214447842785186
Epoch 16/25, Average Loss: 1.2177743955623883
Epoch 17/25, Average Loss: 1.213762878274625
Epoch 18/25, Average Loss: 1.2058143838782984
Epoch 19/25, Average Loss: 1.205070620665521
Epoch 20/25, Average Loss: 1.217559916841472
Epoch 21/25, Average Loss: 1.192050176164124
Epoch 22/25, Average Loss: 1.18902153581198
Epoch 2

## Testing the model on the congressional voting dataset

#### Loading the dataset

In [28]:
cong_voting = pd.read_csv('./preprocessed-datasets/CongressionVoting_prepro.csv')
# encode class value democrat as 1 and republican as 0
cong_voting['class'] = cong_voting['class'].map({'democrat': 1, 'republican': 0})
cong_voting.head()

,ID,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-crporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa,class
0,140,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1
1,383,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1
2,201,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1
3,297,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0
4,309,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0


#### Splitting the dataset into training and testing sets, converting to PyTorch tensors and creating PyTorch DataLoaders

In [14]:
train_X, train_Y, test_X, test_Y = train_test_split(cong_voting, "class", return_torch=True)

dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=False)

dataset = TensorDataset(test_X, test_Y)
test_loader = DataLoader(dataset, batch_size=32, shuffle=False)

#### Creating the model, training and testing

In [15]:
input_size = train_X.shape[1] # number of features in congr voting dataset
NumbOfClasses = 2 # 2 classes in congr voting dataset
learning_rate = 0.01
batch_size = 64
num_epochs = 10
hidden_layer_sizes = [25,30]
activation_function = F.tanh

model = NN(input_size=train_X.shape[1], num_classes=NumbOfClasses, hidden_layer_sizes=hidden_layer_sizes, activation_function=activation_function)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_model(model, train_loader, optimizer, criterion, num_epochs)

accuracy, f1 = check_accuracy(train_loader, model)
print(f"Training set accuracy: {accuracy}, F1 Score: {f1}")
accuracy, f1 = check_accuracy(test_loader, model)
print(f"Test set accuracy: {accuracy}, F1 Score: {f1}")

Epoch 1/10, Average Loss: 0.8204039831956228
Epoch 2/10, Average Loss: 0.6987561484177908
Epoch 3/10, Average Loss: 0.6630964974562327
Epoch 4/10, Average Loss: 0.5736254205306371
Epoch 5/10, Average Loss: 0.4424210687478383


Epoch 6/10, Average Loss: 0.396044726173083
Epoch 7/10, Average Loss: 0.2974789614478747
Epoch 8/10, Average Loss: 0.2279231697320938
Epoch 9/10, Average Loss: 0.22331865628560385
Epoch 10/10, Average Loss: 0.37387768800059956
Training set accuracy: 0.9597701149425287, F1 Score: 0.9592267251527329
Test set accuracy: 0.8604651162790697, F1 Score: 0.8583056478405315


## Testing the model on bank marketing dataset

#### Loading and preparing dataset

In [29]:
bank_marketing = pd.read_csv('./preprocessed-datasets/bank_marketing_prepro.csv')

In [30]:
column_to_move = 'class'

# Move class to the last index
columns = [col for col in bank_marketing.columns if col != column_to_move] + [column_to_move]
bank_marketing = bank_marketing[columns]

bank_marketing.drop('Unnamed: 0', axis=1,inplace=True)

bank_marketing.head()

,age,default,housing,loan,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,...,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,poutcome_failure,poutcome_nonexistent,poutcome_success,class
0,56,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,0,0,0,0,0,0,1,0,0
1,57,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0
2,37,0.0,1.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0
3,40,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,0,0,0,0,0,0,1,0,0
4,56,0.0,0.0,1.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0


In [31]:
bank_marketing.columns

Index(['age', 'default', 'housing', 'loan', 'campaign', 'pdays', 'previous',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'job_blue-collar', 'job_management', 'job_other',
       'job_self-employed', 'job_serivces', 'job_technician',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'poutcome_failure', 'poutcome_nonexistent',
       'poutcome_success', 'class'],
      dtype='object')

#### Splitting the dataset into training and testing sets, converting to PyTorch tensors and creating PyTorch DataLoaders

In [32]:
train_X, train_Y, test_X, test_Y = train_test_split(bank_marketing, "class", return_torch=True)

dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=False)

dataset = TensorDataset(test_X, test_Y)
test_loader = DataLoader(dataset, batch_size=32, shuffle=False)

#### Creating the model, training and testing

In [33]:
input_size = train_X.shape[1] 
NumbOfClasses = 2 
learning_rate = 0.01
batch_size = 64
num_epochs = 10
hidden_layer_sizes = [25,30]
activation_function = F.tanh

model = NN(input_size=train_X.shape[1], num_classes=NumbOfClasses, hidden_layer_sizes=hidden_layer_sizes, activation_function=activation_function)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_model(model, train_loader, optimizer, criterion, num_epochs)

accuracy, f1 = check_accuracy(train_loader, model)
print(f"Training set accuracy: {accuracy}, F1 Score: {f1}")
accuracy, f1 = check_accuracy(test_loader, model)
print(f"Test set accuracy: {accuracy}, F1 Score: {f1}")

Epoch 1/25, Average Loss: 0.3309118606186318


KeyboardInterrupt: 

# Test Grid search over all three datasets

It was not possible for us to iterate over all datasets because of the large amount of runtime

### Bank Marketing

In [15]:
hidden_layer_sizes_list = [[5],[10],[25, 30], [20, 25, 30]]
activation_functions = [F.tanh, F.relu, F.sigmoid]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [64]
num_epochs_list = [10]

dataset = bank_marketing

X = dataset.drop(["class"], axis=1).values
Y = dataset["class"].values
data = TensorDataset(torch.tensor(X), torch.tensor(Y))
data_loader = DataLoader(data, batch_size=32, shuffle=False)

input_size = X.shape[1]

NumbOfClasses = 2

grid_results_bank, best_f1, best_combination = grid_search_cv(
    hidden_layer_sizes_list, activation_functions, learning_rates, batch_sizes, num_epochs_list, data_loader, NumbOfClasses,
    k_folds=5, use_scaling=True)

grid_results_bank['dataset'] = 'bank_marketing'

Epoch 1/10, Average Loss: 0.30895951283209533
Epoch 2/10, Average Loss: 0.28735410527696886
Epoch 3/10, Average Loss: 0.28536031419791064
Epoch 4/10, Average Loss: 0.283694635852448
Epoch 5/10, Average Loss: 0.28263048726378137
Epoch 6/10, Average Loss: 0.28143955136387094
Epoch 7/10, Average Loss: 0.28098883524681756
Epoch 8/10, Average Loss: 0.2809191710306603
Epoch 9/10, Average Loss: 0.2809496916468861
Epoch 10/10, Average Loss: 0.2810058439385544
Epoch 1/10, Average Loss: 0.3128876335412553
Epoch 2/10, Average Loss: 0.29042269303960705
Epoch 3/10, Average Loss: 0.2876893164752756
Epoch 4/10, Average Loss: 0.2868161084727176
Epoch 5/10, Average Loss: 0.28492116279972407
Epoch 6/10, Average Loss: 0.28500315247519503
Epoch 7/10, Average Loss: 0.2842394628137061
Epoch 8/10, Average Loss: 0.2834020783600298
Epoch 9/10, Average Loss: 0.28343992003248736
Epoch 10/10, Average Loss: 0.28344538245096945
Epoch 1/10, Average Loss: 0.3112315728681759
Epoch 2/10, Average Loss: 0.287236833774927

KeyboardInterrupt: 

In [19]:
grid_results_bank

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Average Accuracy,Average Training Time,dataset
0,[5],tanh,0.0100,64,10,0.795699,12.364878,bank_marketing
1,[5],tanh,0.0010,64,10,0.755577,12.158159,bank_marketing
2,[5],tanh,0.0001,64,10,0.718461,11.684069,bank_marketing
3,[5],relu,0.0100,64,10,0.770642,11.399968,bank_marketing
4,[5],relu,0.0010,64,10,0.744679,11.911103,bank_marketing
5,[5],relu,0.0001,64,10,0.731579,11.358967,bank_marketing
6,[5],sigmoid,0.0100,64,10,0.754432,11.386674,bank_marketing
7,[5],sigmoid,0.0010,64,10,0.743364,11.465565,bank_marketing
8,[5],sigmoid,0.0001,64,10,0.735661,11.650078,bank_marketing
9,[10],tanh,0.0100,64,10,0.820842,11.603699,bank_marketing


### Wine quality

In [15]:
hidden_layer_sizes_list = [[5],[10],[25, 30], [20, 25, 30]]
activation_functions = [F.tanh, F.relu, F.sigmoid]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [64]
num_epochs_list = [10]

dataset = wine_quality

#smote_in = True
#train_X, train_Y, test_X, test_Y = train_test_split(dataset, "class", return_torch=True, DoSmote=smote_in)

#train_data = TensorDataset(train_X, train_Y)
#train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

#test_data = TensorDataset(test_X, test_Y)
#test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

X = dataset.drop(["class"], axis=1).values
Y = dataset["class"].values
data = TensorDataset(torch.tensor(X), torch.tensor(Y))
data_loader = DataLoader(data, batch_size=32, shuffle=False)

input_size = X.shape[1]

NumbOfClasses = 10

grid_results_wine, best_f1, best_combination = grid_search_cv(
    hidden_layer_sizes_list, activation_functions, learning_rates, batch_sizes, num_epochs_list, data_loader, NumbOfClasses,
    k_folds=5, use_scaling=True)

grid_results_wine['dataset'] = 'wine_quality'

Epoch 1/10, Average Loss: 2.139833171193193
Epoch 2/10, Average Loss: 2.036203986260949
Epoch 3/10, Average Loss: 2.009973778957274
Epoch 4/10, Average Loss: 1.9749669040121682
Epoch 5/10, Average Loss: 1.948377251625061
Epoch 6/10, Average Loss: 1.9383421961854144
Epoch 7/10, Average Loss: 1.9312357059339198
Epoch 8/10, Average Loss: 1.9251614968951156
Epoch 9/10, Average Loss: 1.9228179382114876
Epoch 10/10, Average Loss: 1.919748586852376
Epoch 1/10, Average Loss: 2.1393754496807005
Epoch 2/10, Average Loss: 2.0301392601757513
Epoch 3/10, Average Loss: 1.999662078008419
Epoch 4/10, Average Loss: 1.9640896393031608
Epoch 5/10, Average Loss: 1.9477385820412054
Epoch 6/10, Average Loss: 1.9411988360125845
Epoch 7/10, Average Loss: 1.9298798936169321
Epoch 8/10, Average Loss: 1.9242776675922115
Epoch 9/10, Average Loss: 1.9181502490508846
Epoch 10/10, Average Loss: 1.9116715294558828
Epoch 1/10, Average Loss: 2.135673844232792
Epoch 2/10, Average Loss: 2.030358060104091
Epoch 3/10, Aver

In [16]:
grid_results_wine


,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Average Accuracy,Average F1,Average Training Time,dataset
0,[5],tanh,0.0100,64,10,0.533015,0.475026,1.834746,wine_quality
1,[5],tanh,0.0010,64,10,0.444663,0.371314,1.880026,wine_quality
2,[5],tanh,0.0001,64,10,0.253193,0.250863,1.958428,wine_quality
3,[5],relu,0.0100,64,10,0.553334,0.520799,1.619479,wine_quality
4,[5],relu,0.0010,64,10,0.473290,0.414571,1.618498,wine_quality
5,[5],relu,0.0001,64,10,0.064952,0.034494,1.565606,wine_quality
6,[5],sigmoid,0.0100,64,10,0.536865,0.465590,1.825684,wine_quality
7,[5],sigmoid,0.0010,64,10,0.498074,0.431516,1.916184,wine_quality
8,[5],sigmoid,0.0001,64,10,0.042635,0.016090,1.879657,wine_quality
9,[10],tanh,0.0100,64,10,0.558259,0.525075,1.944383,wine_quality


### Congressional Voting

In [20]:
hidden_layer_sizes_list = [[5],[10],[25, 30], [20, 25, 30]]
activation_functions = [F.tanh, F.relu, F.sigmoid]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [64]
num_epochs_list = [10]

dataset = cong_voting

#smote_in = True
#train_X, train_Y, test_X, test_Y = train_test_split(dataset, "class", return_torch=True, DoSmote=smote_in)

#train_data = TensorDataset(train_X, train_Y)
#train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

#test_data = TensorDataset(test_X, test_Y)
#test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

X = dataset.drop(["class"], axis=1).values
Y = dataset["class"].values
data = TensorDataset(torch.tensor(X), torch.tensor(Y))
data_loader = DataLoader(data, batch_size=32, shuffle=False)

input_size = X.shape[1]

NumbOfClasses = 2

grid_results_voting, best_accuracy, best_combination = grid_search_cv(
    hidden_layer_sizes_list, activation_functions, learning_rates, batch_sizes, num_epochs_list, data_loader, NumbOfClasses,
    k_folds=5, use_scaling=True)

grid_results_voting['dataset'] = 'cong_voting'

Epoch 1/10, Average Loss: 1.0931169390678406
Epoch 2/10, Average Loss: 0.8803235093752543
Epoch 3/10, Average Loss: 0.7031833132108053
Epoch 4/10, Average Loss: 0.5734289089838663
Epoch 5/10, Average Loss: 0.4860897362232208
Epoch 6/10, Average Loss: 0.40965110063552856
Epoch 7/10, Average Loss: 0.36114127437273663
Epoch 8/10, Average Loss: 0.32630948225657147
Epoch 9/10, Average Loss: 0.2915192147095998
Epoch 10/10, Average Loss: 0.2733388940493266
Epoch 1/10, Average Loss: 1.104884922504425
Epoch 2/10, Average Loss: 0.8754033048947653
Epoch 3/10, Average Loss: 0.7022010485331217
Epoch 4/10, Average Loss: 0.5597648223241171
Epoch 5/10, Average Loss: 0.46973538398742676
Epoch 6/10, Average Loss: 0.3878113627433777
Epoch 7/10, Average Loss: 0.32780036330223083
Epoch 8/10, Average Loss: 0.2840626736481984
Epoch 9/10, Average Loss: 0.26743921140829724
Epoch 10/10, Average Loss: 0.24184114237626395
Epoch 1/10, Average Loss: 1.102731962998708
Epoch 2/10, Average Loss: 0.8889010945955912
Epo

KeyboardInterrupt: 

In [23]:
grid_results_voting

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Average Accuracy,Average Training Time,dataset
0,[5],tanh,0.0100,64,10,0.896471,0.141268,cong_voting
1,[5],tanh,0.0010,64,10,0.287731,0.122248,cong_voting
2,[5],tanh,0.0001,64,10,0.212773,0.124714,cong_voting
3,[5],relu,0.0100,64,10,0.890756,0.126250,cong_voting
4,[5],relu,0.0010,64,10,0.304706,0.119831,cong_voting
5,[5],relu,0.0001,64,10,0.235798,0.114162,cong_voting
6,[5],sigmoid,0.0100,64,10,0.913782,0.058026,cong_voting
7,[5],sigmoid,0.0010,64,10,0.264706,0.057321,cong_voting
8,[5],sigmoid,0.0001,64,10,0.166891,0.054440,cong_voting
9,[10],tanh,0.0100,64,10,0.936639,0.057579,cong_voting


#### Merging

In [19]:
full_results_df = pd.concat([grid_results_voting,grid_results_wine,grid_results_bank], ignore_index=True)
full_results_df

NameError: name 'grid_results_voting' is not defined

In [18]:
full_results_df.to_csv('./results/cv_grid_search_results.csv', index=False)

NameError: name 'full_results_df' is not defined

### Top Perfroming Configurations

In [17]:
top_models_rows = []

for dataset in full_results_df['dataset'].unique():
    top_models_rows.extend(full_results_df[full_results_df['dataset'] == dataset].nlargest(2, 'Average Accuracy').iterrows())

top_models_rows_data = [row[1] for row in top_models_rows]

top_models_df = pd.DataFrame(top_models_rows_data).reset_index(drop=True)

top_models_df

NameError: name 'full_results_df' is not defined